当我们添加通道时，我们的输入和隐藏的表示都变成了三维张量。例如，每个RGB输入图像具有 3×h×w 的形状。我们将这个大小为 3 的轴称为通道（channel）维度。在本节中，我们将更深入地研究具有多输入和多输出通道的卷积核。
B, C, W, H

### 6.4.1. 多输入通道
当输入 包含多个通道时，
##### 需要构造一个与输入数据具有相同通道数的卷积核。

假设输入的通道数为 $c_i$ ，那么卷积核的输入通道数也需要为 $c_i$ 。如果卷积核的窗口形状是 $k_h×k_w$ ，那么当 $c_i=1$ 时，我们可以把卷积核看作形状为 $k_h×k_w$ 的二维张量。

然而，当$c_i > 1$时，我们卷积核的每个输入通道，将包含 $k_h×k_w$ 的张量。
把这 $c_i$个张量连结起来可以得到形状为  $c_i × k_h × k_w$的卷积核。
由于输入和卷积核都有$c_i$个通道，
##### 我们可以对每个通道输入的二维张量进行互相关操作，
#### 然后再对通道求和（将$c_i$的结果相加）得到二维张量。
这是多通道输入和多输入通道卷积核之间进行二维互相关运算的结果。
![多通道卷积](imgs/6_4_1多输入通道1.png)

为了加深理解，我们实现一下多输入通道互相关运算。 简而言之，我们所做的就是对每个通道执行互相关操作，然后将结果相加。

In [12]:
import torch
from d2l import torch as d2l

def corr2d_multi_in(X, K):
    # 先遍历“X”和“K”的第0个维度（通道维度），再把它们加在一起
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))

In [13]:
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
               [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

In [14]:
print(K.shape)

torch.Size([2, 2, 2])


In [15]:
print(X.shape)

torch.Size([2, 3, 3])


### 6.4.2. 多输出通道
到目前为止，不论有多少输入通道，我们还只有一个输出通道。
然而，每一层有多个输出通道是至关重要的。

在最流行的神经网络架构中，随着神经网络层数的加深，我们常会增加输出通道的维数，通过减少空间分辨率以获得更大的通道深度。

直观地说，我们可以将每个通道看作是对不同特征的响应。
而现实可能更为复杂一些，因为每个通道不是独立学习的，而是为了共同使用而优化的。
因此，多输出通道并不仅是学习多个单通道的检测器。

用$c_i$和$c_o$表述输入和输出通道的数目，
并$k_h$和$k_w$表示卷积核的高度和宽度

为了获得多个通道的输出，我们可以
##### 为每个输出通道创建一个$c_i * k_h * k_w$的卷积核张量，
这样卷积核的形状是$c_o * c_i * k_h * k_w$

在互相关运算中，每个输出通道先获取所有输入通道，再以对应该输出通道的卷积核计算出结果。
如下所示，我们实现一个计算多个通道的输出的互相关函数。

In [16]:
def corr2d_multi_in_out(X, K):
    # 迭代“K”的第0个维度，每次都对输入“X”执行互相关运算。
    # 最后将所有结果都叠加在一起
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

In [17]:
K = torch.stack((K, K + 1, K + 2), 0)
K.shape

torch.Size([3, 2, 2, 2])

In [18]:
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

#### 6.4.3.  1×1  卷积层
1×1 卷积，即 $k_h=k_w=1$ ，看起来似乎没有多大意义。
毕竟，卷积的本质是有效提取相邻像素间的相关特征，而 1×1 卷积显然没有此作用。
尽管如此， 1×1 仍然十分流行，时常包含在复杂深层网络的设计中。
因为使用了最小窗口，1x1卷积失去了卷积的特有能力--在高度和宽度维度上，识别相邻元素间相互作用的能力。

##### 其实 1×1 卷积的唯一计算发生在通道上。
![1x1卷积层](imgs/6_4_31x1卷积层1.png)

In [19]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    # 全连接层中的矩阵乘法
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))

In [20]:
X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
assert float(torch.abs(Y1 - Y2).sum()) < 1e-6